# QCArchive+QCMLForge Demo with CyberShuttle

The first half of this demo shows how to use QCArchive to setup a dataset
and run computations with ease. The compute resource for this noteboook
uses Cybershuttle; however, a purely local resource demo is available under
`./demo_local.ipynb`.

The second half of this demo shows how one can consume the generated data
to train AP-Net models through QCMLForge. 

## How is this useful?
Prior to using quantum mechanical methods for specific applications, often
computational chemists will either consult previous studies of benchmarked
methods on systems similar to the systems of interest, or perform the
benchmarking task themselves. Then after knowing how much error to expect,
the level of theory that is the most inexpensive, but still within reasonable
error estimates will be selected for scaling up to novel system(s).

The Sherrill research group has developed a reputation for using quantum
mechanical (QM) methods to study intermolecular interaction energies to basically
determine how attractive (or repulsive) molecules behave when brought close
together. Before applying QM methods to novel systems, oftentimes these
methods need to be benchmarked to better understand what methods should be
used for studying particular systems. As such, the Sherrill group has performed
many benchmarking studies on interaction energies over the years to investigate
what methods (HF, MP2, CCSD, CCSD(T), DFT, SAPT, etc.) give reliable interaction
energies that can be used in various downstream applications. The first half of 
this notebook demonstrates how one can easily perform such benchmarking tasks through
QCFractal/QCArchive/Cybershuttle software. 

In [1]:
!pip install --force-reinstall -q "airavata-python-sdk[notebook]"
import airavata_jupyter_magic

  DEPRECATION: Building 'pysftp' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pysftp'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'thrift' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'thrift'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'thrift_connector' using the legacy setup.py bdist_wheel mechani

In [2]:
%authenticate

Output()

Authenticated.

In [ ]:
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=expanse:shared
%wait_for_runtime hpc_cpu
%switch_runtime hpc_cpu

Requesting runtime=hpc_cpu
cpuCount: 32
experimentName: CS_Agent
group: Default
libraries:
- numpy=2.2.5
- pandas=2.2.3
- pip
- psycopg2=2.9.9
- postgresql=17.4
- pytest
- python=3.10
- psi4=1.9.1
- pytorch-cpu=2.5.1
- jupyter=1.1.1
- requests
- setuptools
- torchaudio=2.5.1
- torchvision=0.20.1
- pytorch_geometric=2.6.1
- pytorch_scatter=2.1.2=cpu*
- pytorch-minimize=0.0.2
- matplotlib=3.10.1
- pydantic=1
- scipy=1.15.*
- tqdm
memory: 64000
mounts:
- cybershuttle-reference:/cybershuttle_data/cybershuttle-reference
nodeCount: 1
pip:
- cdsg-tools==0.0.2
- qm-tools-aw==1.4.5
- qcmlforge==0.0.6
- qcfractal==0.59
- qcmanybody
- qcfractalcompute==0.59
queue: shared
remoteCluster: expanse
wallTime: 60

Requested runtime=hpc_cpu. state=CONFIGURING_WORKSPACE
Switched to runtime=hpc_cpu.


In [4]:
%copy_data source=local:setup_qcfractal.py target=hpc_cpu:setup_qcfractal.py
%copy_data source=local:__init__.py target=hpc_cpu:__init__.py
%copy_data source=local:combined_df_subset_358.pkl target=hpc_cpu:combined_df_subset_358.pkl

Copying from local:setup_qcfractal.py to hpc_cpu:setup_qcfractal.py
Pushing local:setup_qcfractal.py to remote:setup_qcfractal.py
[200] Uploaded local:setup_qcfractal.py to remote:setup_qcfractal.py
Copying from local:__init__.py to hpc_cpu:__init__.py
Pushing local:__init__.py to remote:__init__.py
[200] Uploaded local:__init__.py to remote:__init__.py
Copying from local:combined_df_subset_358.pkl to hpc_cpu:combined_df_subset_358.pkl
Pushing local:combined_df_subset_358.pkl to remote:combined_df_subset_358.pkl
[200] Uploaded local:combined_df_subset_358.pkl to remote:combined_df_subset_358.pkl


In [106]:
import psi4
from pprint import pprint as pp
import pandas as pd
import numpy as np
from qm_tools_aw import tools
import matplotlib.pyplot as plt
# QCElemental Imports
from qcelemental.models import Molecule
import qcelemental as qcel
# Dataset Imports
from qcportal import PortalClient
from qcportal.singlepoint import SinglepointDatasetEntry, QCSpecification
from qcportal.manybody import ManybodyDatasetEntry, ManybodySpecification
from torch import manual_seed

manual_seed(42)

h2kcalmol = qcel.constants.hartree2kcalmol
print('Imports')

⠧ Connecting to=hpc_cpu... status=CONNECTED
Imports


# QCArchive Setup

In [7]:
from setup_qcfractal import setup_qcarchive_qcfractal
import os

# Update these if you request non-default resources from cybershuttle.yml
max_workers = 3
cores_per_worker = 8
memory_per_worker = 16

setup_qcarchive_qcfractal(
    QCF_BASE_FOLDER=os.path.join(os.getcwd(), "qcfractal"),
    start=False,
    reset=False,
    db_config={
        "name": None,
        "enable_security": "false",
        "allow_unauthenticated_read": None,
        "logfile": None,
        "loglevel": None,
        "service_frequency": 5,
        "max_active_services": None,
        "heartbeat_frequency": 60,
        "log_access": None,
        "database": {
            "base_folder": None,
            "host": None,
            "port": 5433,
            "database_name": "qca",
            "username": None,
            "password": None,
            "own": None,
        },
        "api": {
            "host": None,
            "port": 7778,
            "secret_key": None,
            "jwt_secret_key": None,
        },
    },
    resources_config={
            "update_frequency": 5,
            "cores_per_worker": cores_per_worker,
            "max_workers": max_workers,
            "memory_per_worker": memory_per_worker,
    },
    conda_env=None,
    worker_sh=None,
)

⠧ Connecting to=hpc_cpu... status=CONNECTED
/workspace/qcfractal
--------------------------------------------------------------------------------
Python executable:  /dev/shm/cybershuttle/envs/18dccede/bin/python3.10
QCFractal version:  0.59
QCFractal alembic revision:  d5988aa750ae
pg_ctl path:  /dev/shm/cybershuttle/envs/18dccede/bin/pg_ctl
PostgreSQL server version:  PostgreSQL 17.4 on x86_64-conda-linux-gnu, compiled by x86_64-conda-linux-gnu-cc (conda-forge gcc 13.3.0-2) 13.3.0, 64-bit
--------------------------------------------------------------------------------


Displaying QCFractal configuration below
--------------------------------------------------------------------------------
access_log_keep: 0
allow_unauthenticated_read: true
api:
  extra_flask_options: null
  extra_waitress_options: null
  host: localhost
  jwt_access_token_expires: 3600
  jwt_refresh_token_expires: 86400
  jwt_secret_key: kKQrB4Hsms_waPwy70QsGftZz_64HqR9VLItMkFA7rU
  num_threads_per_worker: 4
  port:

In [8]:
get_ipython().system = os.system
!qcfractal-server --config=`pwd`/qcfractal/qcfractal_config.yaml start > qcfractal/qcf_server.log &

⠧ Connecting to=hpc_cpu... status=CONNECTED
0


In [9]:
!qcfractal-compute-manager --config=`pwd`/qcfractal/resources.yml &

⠧ Connecting to=hpc_cpu... status=CONNECTED
0


# QCArchive single point example

In [10]:
# Establish client connection
client = PortalClient("http://localhost:7778", verify=False)

⠧ Connecting to=hpc_cpu... status=CONNECTED
[2025-05-01 07:17:27 PDT]     INFO: qcfractalcompute.config: Reading configuration data from /workspace/qcfractal/resources.yml


In [16]:
# Running a single job
client = PortalClient("http://localhost:7778", verify=False)
mol = Molecule.from_data(
    """
     0 1
     O  -1.551007  -0.114520   0.000000
     H  -1.934259   0.762503   0.000000
     H  -0.599677   0.040712   0.000000
     --
     0 1
     O   1.350625   0.111469   0.000000
     H   1.680398  -0.373741  -0.758561
     H   1.680398  -0.373741   0.758561

     units angstrom
     no_reorient
     symmetry c1
"""
)

psi4.set_options(
    {"basis": "aug-cc-pvdz", "scf_type": "df", "e_convergence": 6, "freeze_core": True}
)

client.add_singlepoints(
    [mol],
    "psi4",
    driver="energy",
    method="b3lyp",
    basis="aug-cc-pvdz",
    keywords={"scf_type": "df", "e_convergence": 6, "freeze_core": True},
    tag="local",
)
recs = client.query_records(
    record_id=[1]
)
for rec in recs:
    print(rec.dict)
    print(rec.properties['return_result'])

# Can print records
# for rec in client.query_records():
#     pp(rec.dict)
#     pp(rec.error)

⠧ Connecting to=hpc_cpu... status=CONNECTED
<bound method BaseModel.dict of SinglepointRecord(id=1, record_type='singlepoint', is_service=False, properties={'pe energy': 0.0, 'scf dipole': [1.0144337612148102, 0.030397167953601256, 4.72047227695338e-12], 'calcinfo_nmo': 82, 'dft xc energy': -15.093933325834776, 'return_energy': -152.8965873433082, 'return_result': -152.8965873433082, 'scf_xc_energy': -15.093933325834776, 'calcinfo_natom': 6, 'calcinfo_nbeta': 10, 'current dipole': [1.0144337612148102, 0.030397167953601256, 4.72047227695338e-12], 'current energy': -152.8965873433082, 'return_hessian': None, 'scf_iterations': 8, 'calcinfo_nalpha': 10, 'calcinfo_nbasis': 82, 'dft vv10 energy': 0.0, 'return_gradient': None, 'dft total energy': -152.8965873433082, 'scf_total_energy': -152.8965873433082, 'scf_dipole_moment': [1.0144337612148102, 0.030397167953601256, 4.72047227695338e-12], 'scf_total_hessian': None, 'scf total energies': [-152.1832917902241, -152.54809853364756, -151.7847704

# QCArchive dataset examples

In [ ]:
# Creating a QCArchive Dataset...
# Load in a dataset from a recent Sherrill work (Levels of SAPT II)
df_LoS = pd.read_pickle("./combined_df_subset_358.pkl")
print(df_LoS[['Benchmark', 'SAPT2+3(CCD)DMP2 TOTAL ENERGY aqz', 'MP2 IE atz', 'SAPT0 TOTAL ENERGY adz' ]])

# Limit to 100 molecules with maximum of 16 atoms to keep computational cost down
df_LoS['size'] = df_LoS['atomic_numbers'].apply(lambda x: len(x))
df_LoS = df_LoS[df_LoS['size'] <= 16]
df_LoS = df_LoS.sample(100, random_state=42, axis=0).copy()
# df_LoS = df_LoS.sample(50, random_state=42, axis=0).copy()
df_LoS.reset_index(drop=True, inplace=True)
print(df_LoS['size'].describe())

# Create QCElemntal Molecules to generate the dataset
def qcel_mols(row):
    """
    Convert the row to a qcel molecule
    """
    atomic_numbers = [row['atomic_numbers'][row['monAs']], row['atomic_numbers'][row['monBs']]]
    coords = [row['coordinates'][row['monAs']], row['coordinates'][row['monBs']]]
    cm = [
        [row['monA_charge'], row['monA_multiplicity']],
        [row['monB_charge'], row['monB_multiplicity']],
     ]
    return tools.convert_pos_carts_to_mol(atomic_numbers, coords, cm)
df_LoS['qcel_molecule'] = df_LoS.apply(qcel_mols, axis=1)
geoms = df_LoS['qcel_molecule'].tolist()
ref_IEs = df_LoS['Benchmark'].tolist()
sapt0_adz = (df_LoS['SAPT0 TOTAL ENERGY adz'] * h2kcalmol).tolist()

⠧ Connecting to=hpc_cpu... status=CONNECTED
Benchmark SAPT2+3(CCD)DMP2 TOTAL ENERGY aqz  MP2 IE atz  \
0      -10.248                         -0.016681   -0.015629   
1      -15.245                         -0.024763   -0.023012   
2       -3.517                         -0.005637   -0.005608   
3       -0.127                         -0.000187   -0.000194   
4       -8.990                         -0.014655   -0.013687   
..         ...                               ...         ...   
353     -4.390                         -0.007196   -0.006835   
354     -1.130                         -0.001489   -0.002395   
355     -0.260                         -0.000432   -0.000450   
356     -5.740                         -0.009198   -0.008974   
357     -3.120                         -0.004909   -0.005518   

     SAPT0 TOTAL ENERGY adz  
0                 -0.018254  
1                 -0.027620  
2                 -0.005920  
3                 -0.000192  
4                 -0.016209  
..          

## Singlepoint Dataset

In [18]:
# Create client dataset

ds_name = 'S22-singlepoint'
client_datasets = [i['dataset_name'] for i in client.list_datasets()]
# Check if dataset already exists, if not create a new one
if ds_name not in client_datasets:
    ds = client.add_dataset("singlepoint", ds_name,
                            f"Dataset to contain {ds_name}")
    print(f"Added {ds_name} as dataset")
    # Insert entries into dataset
    entry_list = []
    for idx, mol in enumerate(geoms):
        extras = {
            "name": 'S22-' + str(idx),
            "idx": idx,
        }
        mol = Molecule.from_data(mol.dict(), extras=extras)
        ent = SinglepointDatasetEntry(name=extras['name'], molecule=mol)
        entry_list.append(ent)
    ds.add_entries(entry_list)
    print(f"Added {len(entry_list)} molecules to dataset")
else:
    ds = client.get_dataset("singlepoint", ds_name)
    print(f"Found {ds_name} dataset, using this instead")

print(ds)

⠇ Connecting to=hpc_cpu... status=CONNECTED
Added S22-singlepoint as dataset
Added 150 molecules to dataset
id=1 dataset_type='singlepoint' name='S22-singlepoint' description='Dataset to contain S22-singlepoint' tagline='' tags=[] group='default' visibility=True provenance={} default_tag='*' default_priority=<PriorityEnum.normal: 1> owner_user=None owner_group=None metadata={} extras={} contributed_values_=None attachments_=None auto_fetch_missing=True


In [19]:
# Can delete the dataset if you want to start over. Need to know dataset_id
# client.delete_dataset(dataset_id=ds.id, delete_records=True)

⠧ Connecting to=hpc_cpu... status=CONNECTED


In [20]:
# SAPT0 Example
method, basis = "SAPT0", "cc-pvdz"

# Set the QCSpecification (QM interaction energy in our case)
spec = QCSpecification(
    program="psi4",
    driver="energy",
    method=method,
    basis=basis,
    keywords={
        "scf_type": "df",
    },
)
ds.add_specification(name=f"psi4/{method}/{basis}", specification=spec)

⠧ Connecting to=hpc_cpu... status=CONNECTED
InsertMetadata(error_description=None, errors=[], inserted_idx=[0], existing_idx=[])


In [21]:
# Run the computations
ds.submit()
print(f"Submitted {ds_name} dataset")

⠧ Connecting to=hpc_cpu... status=CONNECTED
Submitted S22-singlepoint dataset


In [22]:
# Check the status of the dataset - can repeatedly run this to see the progress
ds.status()

⠧ Connecting to=hpc_cpu... status=CONNECTED
{'psi4/SAPT0/cc-pvdz': {<RecordStatusEnum.waiting: 'waiting'>: 150}}


## Manybody Dataset

In [23]:
# Create client dataset
ds_name_mb = 'S22-manybody'
client_datasets = [i['dataset_name'] for i in client.list_datasets()]
# Check if dataset already exists, if not create a new one
if ds_name_mb not in client_datasets:
    print("Setting up new dataset:", ds_name_mb)
    ds_mb = client.add_dataset("manybody", ds_name_mb,
                            f"Dataset to contain {ds_name_mb}")
    print(f"Added {ds_name_mb} as dataset")
    # Insert entries into dataset
    entry_list = []
    for idx, mol in enumerate(geoms):
        ent = ManybodyDatasetEntry(name=f"S22-IE-{idx}", initial_molecule=mol)
        entry_list.append(ent)
    ds_mb.add_entries(entry_list)
    print(f"Added {len(entry_list)} molecules to dataset")
else:
    ds_mb = client.get_dataset("manybody", ds_name_mb)
    print(f"Found {ds_name_mb} dataset, using this instead")

print(ds_mb)

# Can delete the dataset if you want to start over. Need to know dataset_id
# client.delete_dataset(dataset_id=2, delete_records=True)

⠧ Connecting to=hpc_cpu... status=CONNECTED
Setting up new dataset: S22-manybody
Added S22-manybody as dataset
Added 150 molecules to dataset
id=2 dataset_type='manybody' name='S22-manybody' description='Dataset to contain S22-manybody' tagline='' tags=[] group='default' visibility=True provenance={} default_tag='*' default_priority=<PriorityEnum.normal: 1> owner_user=None owner_group=None metadata={} extras={} contributed_values_=None attachments_=None auto_fetch_missing=True


In [24]:
ds_mb.status()

⠧ Connecting to=hpc_cpu... status=CONNECTED
{}


In [ ]:
# Set multiple levels of theory - you can add/remove levels as you desire.
# Computational scaling will get quite expensive with better methods and larger
# basis sets

methods = [
    'hf', # 'svwn', # 'pbe', 
]
basis_sets = [
    '6-31g*'
]

for method in methods:
    for basis in basis_sets:
        # Set the QCSpecification (QM interaction energy in our case)
        qc_spec_mb = QCSpecification(
            program="psi4",
            driver="energy",
            method=method,
            basis=basis,
            keywords={
                "d_convergence": 8,
                "scf_type": "df",
            },
        )

        spec_mb = ManybodySpecification(
            program='qcmanybody',
            bsse_correction=['cp'],
            levels={
                1: qc_spec_mb,
                2: qc_spec_mb,
            },
        )
        print("spec_mb", spec_mb)

        ds_mb.add_specification(name=f"psi4/{method}/{basis}", specification=spec_mb)

        # Run the computations
        ds_mb.submit()
        print(f"Submitted {ds_name} dataset")
# Check the status of the dataset - can repeatedly run this to see the progress
ds_mb.status()

⠧ Connecting to=hpc_cpu... status=CONNECTED
spec_mb program='qcmanybody' levels={1: QCSpecification(program='psi4', driver=<SinglepointDriver.energy: 'energy'>, method='hf', basis='6-31g*', keywords={'d_convergence': 8, 'scf_type': 'df'}, protocols=AtomicResultProtocols(wavefunction=<WavefunctionProtocolEnum.none: 'none'>, stdout=True, error_correction=ErrorCorrectionProtocol(default_policy=True, policies=None), native_files=<NativeFilesProtocolEnum.none: 'none'>)), 2: QCSpecification(program='psi4', driver=<SinglepointDriver.energy: 'energy'>, method='hf', basis='6-31g*', keywords={'d_convergence': 8, 'scf_type': 'df'}, protocols=AtomicResultProtocols(wavefunction=<WavefunctionProtocolEnum.none: 'none'>, stdout=True, error_correction=ErrorCorrectionProtocol(default_policy=True, policies=None), native_files=<NativeFilesProtocolEnum.none: 'none'>))} bsse_correction=[<BSSECorrectionEnum.cp: 'cp'>] keywords=ManybodyKeywords(return_total_data=False) protocols={}
Submitted S22-singlepoint d

In [104]:
pp(ds.status())
pp(ds_mb.status())

⠧ Connecting to=hpc_cpu... status=CONNECTED
{'psi4/SAPT0/cc-pvdz': {<RecordStatusEnum.complete: 'complete'>: 150}}
{'psi4/hf/6-31g*': {<RecordStatusEnum.complete: 'complete'>: 150},
 'psi4/pbe/6-31g*': {<RecordStatusEnum.complete: 'complete'>: 150},
 'psi4/svwn/6-31g*': {<RecordStatusEnum.complete: 'complete'>: 150}}


In [64]:
ds_mb.detailed_status()

⠧ Connecting to=hpc_cpu... status=CONNECTED
[('S22-IE-11', 'psi4/hf/6-31g*', <RecordStatusEnum.complete: 'complete'>),
 ('S22-IE-3', 'psi4/hf/6-31g*', <RecordStatusEnum.complete: 'complete'>),
 ('S22-IE-16', 'psi4/hf/6-31g*', <RecordStatusEnum.complete: 'complete'>),
 ('S22-IE-62', 'psi4/hf/6-31g*', <RecordStatusEnum.complete: 'complete'>),
 ('S22-IE-35', 'psi4/hf/6-31g*', <RecordStatusEnum.complete: 'complete'>),
 ('S22-IE-32', 'psi4/hf/6-31g*', <RecordStatusEnum.waiting: 'waiting'>),
 ('S22-IE-37', 'psi4/hf/6-31g*', <RecordStatusEnum.waiting: 'waiting'>),
 ('S22-IE-68', 'psi4/hf/6-31g*', <RecordStatusEnum.waiting: 'waiting'>),
 ('S22-IE-10', 'psi4/hf/6-31g*', <RecordStatusEnum.waiting: 'waiting'>),
 ('S22-IE-39', 'psi4/hf/6-31g*', <RecordStatusEnum.waiting: 'waiting'>),
 ('S22-IE-121', 'psi4/hf/6-31g*', <RecordStatusEnum.waiting: 'waiting'>),
 ('S22-IE-105', 'psi4/hf/6-31g*', <RecordStatusEnum.waiting: 'waiting'>),
 ('S22-IE-58', 'psi4/hf/6-31g*', <RecordStatusEnum.waiting: 'waiting'

In [ ]:
pp(ds)
pp(ds_mb)
pp(ds_mb.computed_properties)

# Data Assembly

While you can execute the following blocks before all computations are complete, it is recommended to wait until all computations are complete to continue.

In [75]:
# Singlepoint data assemble
def assemble_singlepoint_data(record):
    record_dict = record.dict()
    qcvars = record_dict["properties"]
    sapt_energies = np.array([np.nan, np.nan, np.nan, np.nan, np.nan])
    sapt_energies[0] = qcvars['sapt total energy']
    sapt_energies[1] = qcvars['sapt elst energy']
    sapt_energies[2] = qcvars['sapt exch energy']
    sapt_energies[3] = qcvars['sapt ind energy']
    sapt_energies[4] = qcvars['sapt disp energy']
    return (
        record.molecule,
        record.molecule.atomic_numbers,
        record.molecule.geometry * qcel.constants.bohr2angstroms,
        int(record.molecule.molecular_charge),
        record.molecule.molecular_multiplicity,
        sapt_energies,
    )

def assemble_singlepoint_data_value_names():
    return [
        'qcel_molecule',
        "Z",
        "R",
        "TQ",
        "molecular_multiplicity",
        "SAPT Energies",
    ]

df = ds.compile_values(
    value_call=assemble_singlepoint_data,
    value_names=assemble_singlepoint_data_value_names(),
    unpack=True,
)
pp(df.columns.tolist())
df_sapt0 = df['psi4/SAPT0/cc-pvdz']

⠧ Connecting to=hpc_cpu... status=CONNECTED
[('psi4/SAPT0/cc-pvdz', 'qcel_molecule'),
 ('psi4/SAPT0/cc-pvdz', 'Z'),
 ('psi4/SAPT0/cc-pvdz', 'R'),
 ('psi4/SAPT0/cc-pvdz', 'TQ'),
 ('psi4/SAPT0/cc-pvdz', 'molecular_multiplicity'),
 ('psi4/SAPT0/cc-pvdz', 'SAPT Energies')]


In [76]:
def assemble_data(record):
    record_dict = record.dict()
    qcvars = record_dict["properties"]
    CP_IE = qcvars['results']['cp_corrected_interaction_energy'] * h2kcalmol
    NOCP_IE = qcvars['results'].get('nocp_corrected_interaction_energy', np.nan) * h2kcalmol
    return (
    record.initial_molecule,
    CP_IE,
    NOCP_IE,
    record.initial_molecule.atomic_numbers,
    record.initial_molecule.geometry * qcel.constants.bohr2angstroms,
    int(record.initial_molecule.molecular_charge),
    record.initial_molecule.molecular_multiplicity,
    )

def assemble_data_value_names():
    return [
        'qcel_molecule',
        "CP_IE",
        "NOCP_IE",
        "Z",
        "R",
        "TQ",
        "molecular_multiplicity"
    ]

df_mb = ds_mb.compile_values(
    value_call=assemble_data,
    value_names=assemble_data_value_names(),
    unpack=True,
)

pp(df_mb.columns.tolist())

⠧ Connecting to=hpc_cpu... status=CONNECTED
[('psi4/hf/6-31g*', 'qcel_molecule'),
 ('psi4/pbe/6-31g*', 'qcel_molecule'),
 ('psi4/hf/6-31g*', 'CP_IE'),
 ('psi4/pbe/6-31g*', 'CP_IE'),
 ('psi4/hf/6-31g*', 'NOCP_IE'),
 ('psi4/pbe/6-31g*', 'NOCP_IE'),
 ('psi4/hf/6-31g*', 'Z'),
 ('psi4/pbe/6-31g*', 'Z'),
 ('psi4/hf/6-31g*', 'R'),
 ('psi4/pbe/6-31g*', 'R'),
 ('psi4/hf/6-31g*', 'TQ'),
 ('psi4/pbe/6-31g*', 'TQ'),
 ('psi4/hf/6-31g*', 'molecular_multiplicity'),
 ('psi4/pbe/6-31g*', 'molecular_multiplicity')]


In [86]:
from cdsg_plot import error_statistics

df_sapt0['sapt0 total energes'] = df_sapt0.apply(lambda x: x['SAPT Energies'][0] * h2kcalmol, axis=1)
df_plot = pd.DataFrame(
    {
        "qcel_molecule": df_mb["psi4/pbe/6-31g*"]["qcel_molecule"],
        "HF/6-31G*": df_mb["psi4/hf/6-31g*"]["CP_IE"],
        'SAPT0/cc-pvdz': df_sapt0['sapt0 total energes'].values,
        # "svwn/6-31G*": df_mb["psi4/svwn/6-31g*"]["CP_IE"],
        # "PBE/6-31G*": df_mb["psi4/pbe/6-31g*"]["CP_IE"],
    }
)
# print(df_plot)
id = [int(i[7:]) for i in df_plot.index]
df_plot['id'] = id
df_plot.sort_values(by='id', inplace=True, ascending=True)
df_plot['reference'] = ref_IEs
df_plot['SAPT0/aug-cc-pvdz'] = sapt0_adz
df_plot['HF/6-31G* error'] = (df_plot['HF/6-31G*'] - df_plot['reference']).astype(float)
# df_plot['PBE/6-31G* error'] = (df_plot['PBE/6-31G*'] - df_plot['reference']).astype(float)
# df_plot['svwn/6-31G* error'] = (df_plot['svwn/6-31G*'] - df_plot['reference']).astype(float)
df_plot['SAPT0/cc-pvdz error'] = (df_plot['SAPT0/cc-pvdz'] - df_plot['reference']).astype(float)
df_plot['SAPT0/aug-cc-pvdz error'] = (df_plot['SAPT0/aug-cc-pvdz'] - df_plot['reference']).astype(float)
# print(df_plot[['HF/6-31G*', 'SAPT0/cc-pvdz', 'reference', "SAPT0/aug-cc-pvdz"]])
df_plot = df_plot.dropna(subset=['qcel_molecule', 'HF/6-31G*', 'SAPT0/cc-pvdz', 'SAPT0/aug-cc-pvdz'])
print(df_plot[['HF/6-31G* error', 'SAPT0/cc-pvdz error', "SAPT0/aug-cc-pvdz error"]].describe())

⠇ Connecting to=hpc_cpu... status=CONNECTED
HF/6-31G* error  SAPT0/cc-pvdz error  SAPT0/aug-cc-pvdz error
count        86.000000            86.000000                86.000000
mean          1.800908            -0.355842                -1.253967
std           2.245827             1.064127                 1.733738
min          -1.051059            -3.507544                -6.354809
25%           0.149398            -0.431914                -1.593823
50%           1.064267            -0.019710                -0.481430
75%           2.435488             0.253241                -0.197608
max           9.469009             1.198256                 0.220561


# Plotting the interaction energy errors

In [87]:
error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/cc-pvdz": "SAPT0/cc-pvdz error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
    },
    output_filename="S22-IE.png",
    figure_size=(6, 6),
    x_label_fontsize=16,
    ylim=(-15, 15),
    rcParams={},
    usetex=False,
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

⠧ Connecting to=hpc_cpu... status=CONNECTED
Plotting S22-IE.png
(-15, 15)
lower_bound = -15, upper_bound = 20, inc = 5
S22-IE_violin.png


![S22-IE_violin.png](./S22-IE_violin.png)

# QCMLForge

## AP-Net2 inference

In [88]:
import apnet_pt
from apnet_pt.AtomPairwiseModels.apnet2 import APNet2Model
from apnet_pt.AtomModels.ap2_atom_model import AtomModel

atom_model = AtomModel().set_pretrained_model(model_id=0)
ap2 = APNet2Model(atom_model=atom_model.model).set_pretrained_model(model_id=0)
ap2.atom_model = atom_model.model
print(df_plot['qcel_molecule'].tolist())
apnet2_ies_predicted = ap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=16
)

⠧ Connecting to=hpc_cpu... status=CONNECTED
running on the CPU
running on the CPU
self.dataset=None
[Molecule(name='C3H9NO2', formula='C3H9NO2', hash='8682f32'), Molecule(name='C4H8O4', formula='C4H8O4', hash='35b6c18'), Molecule(name='H4O2', formula='H4O2', hash='80c4390'), Molecule(name='C2H6N2O2', formula='C2H6N2O2', hash='084885d'), Molecule(name='C2H4O4', formula='C2H4O4', hash='f55cc2e'), Molecule(name='CH7NO', formula='CH7NO', hash='682fa43'), Molecule(name='C2H10N2', formula='C2H10N2', hash='9f0ed39'), Molecule(name='C6H8O', formula='C6H8O', hash='849097c'), Molecule(name='CH6O2', formula='CH6O2', hash='5d8f9aa'), Molecule(name='CH6O2', formula='CH6O2', hash='b6e3eb7'), Molecule(name='C5H7NO', formula='C5H7NO', hash='2e03c23'), Molecule(name='C2H5NO3', formula='C2H5NO3', hash='fd88297'), Molecule(name='C2H10N2', formula='C2H10N2', hash='0777887'), Molecule(name='C2H8O2', formula='C2H8O2', hash='9bc507d'), Molecule(name='C2H10N2', formula='C2H10N2', hash='19bb134'), Molecule(nam

In [108]:
# AP-Net2 IE
df_plot['APNet2'] = np.sum(apnet2_ies_predicted, axis=1)
df_plot['APNet2 error'] = (df_plot['APNet2'] - df_plot['reference']).astype(float)
#print(df_plot.sort_values(by='APNet2 error', ascending=True)[['APNet2', 'reference']])
error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/cc-pvdz": "SAPT0/cc-pvdz error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
    },
    output_filename="S22-IE-AP2.png",
    rcParams={},
    usetex=False,
    figure_size=(4, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)
plt.show()

⠇ Connecting to=hpc_cpu... status=CONNECTED
Plotting S22-IE-AP2.png
S22-IE-AP2_violin.png


![S22-IE-AP2_violin.png](./S22-IE-AP2_violin.png)

In [91]:
# Training models on new QM data: Transfer Learning

from apnet_pt import pairwise_datasets

ds2 = pairwise_datasets.apnet2_module_dataset(
    root="data_dir",
    spec_type=None,
    atom_model=atom_model,
    qcel_molecules=df_plot['qcel_molecule'].tolist(),
    energy_labels=[np.array([i]) for i in df_plot['reference'].tolist()],
    skip_compile=True,
    force_reprocess=True,
    atomic_batch_size=8,
    prebatched=False,
    in_memory=True,
    batch_size=4,
)
print(ds2)

⠧ Connecting to=hpc_cpu... status=CONNECTED
Received 86 QCElemental molecules with energy labels
Processing directly from provided QCElemental molecules...
Processing 86 dimers from provided QCElemental molecules...
Creating data objects...
len(RAs)=86, self.atomic_batch_size=8, self.batch_size=4
0/86, 0.04s, 0.04s
8/86, 0.03s, 0.07s


16/86, 0.03s, 0.10s
24/86, 0.03s, 0.13s
32/86, 0.03s, 0.16s
40/86, 0.03s, 0.19s
48/86, 0.03s, 0.22s
56/86, 0.03s, 0.25s
64/86, 0.03s, 0.28s
72/86, 0.02s, 0.31s
80/86, 0.03s, 0.33s
Processing directly from provided QCElemental molecules...
Processing 86 dimers from provided QCElemental molecules...
Creating data objects...
len(RAs)=86, self.atomic_batch_size=8, self.batch_size=4
0/86, 0.03s, 0.03s
8/86, 0.03s, 0.07s


16/86, 0.03s, 0.10s
24/86, 0.03s, 0.13s
32/86, 0.03s, 0.16s
40/86, 0.03s, 0.19s
48/86, 0.03s, 0.23s
56/86, 0.03s, 0.26s
64/86, 0.03s, 0.29s
72/86, 0.03s, 0.32s
80/86, 0.03s, 0.34s
self.root='data_dir', self.spec_type=None, self.in_memory=True
apnet2_module_dataset(86)


## Transfer Learning

In [92]:
# Transfer Learning APNet2 model on computed QM data
ap2.train(
    dataset=ds2,
    n_epochs=50,
    transfer_learning=True,
    skip_compile=True,
    model_path="apnet2_transfer_learning.pt",
    split_percent=0.8,
)

⠧ Connecting to=hpc_cpu... status=CONNECTED
Saving training results to...
apnet2_transfer_learning.pt
~~ Training APNet2Model ~~
Training on 68 samples, Testing on 18 samples

Network Hyperparameters:
self.model.n_message=3
self.model.n_neuron=128
self.model.n_embed=8
self.model.n_rbf=8
self.model.r_cut=5.0
self.model.r_cut_im=8.0
Training Hyperparameters:
n_epochs=50
lr=0.0005
lr_decay=None
batch_size=4
Running single-process training
Total
(Pre-training) (0.93   s)  MAE:   1.607/1.275
EPOCH:    0 (1.04   s)  MAE:   2.208/0.895   *
EPOCH:    1 (0.99   s)  MAE:   1.030/0.731   *
EPOCH:    2 (0.99   s)  MAE:   0.631/0.731   *
EPOCH:    3 (0.99   s)  MAE:   0.490/0.718
EPOCH:    4 (0.97   s)  MAE:   0.388/0.665
EPOCH:    5 (0.99   s)  MAE:   1.010/0.878
EPOCH:    6 (0.96   s)  MAE:   0.742/0.721
EPOCH:    7 (1.14   s)  MAE:   0.774/0.619   *
EPOCH:    8 (1.02   s)  MAE:   0.923/0.865
EPOCH:    9 (1.00   s)  MAE:   0.679/0.645
EPOCH:   10 (0.99   s)  MAE:   0.636/0.460
EPOCH:   11 (0.95  

In [93]:
# AP-Net2 IE
apnet2_ies_predicted_transfer = ap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=16,
)
df_plot['APNet2 transfer'] = np.sum(apnet2_ies_predicted_transfer, axis=1)
df_plot['APNet2 transfer error'] = (df_plot['APNet2 transfer'] - df_plot['reference']).astype(float)

error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
        "APNet2 transfer": "APNet2 transfer error",
    },
    output_filename="S22-IE-AP2-tf.png",
    rcParams={},
    usetex=False,
    figure_size=(6, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

⠧ Connecting to=hpc_cpu... status=CONNECTED
Plotting S22-IE-AP2-tf.png
S22-IE-AP2-tf_violin.png


![S22-IE_violin-AP2-tf.png](./S22-IE-AP2-tf_violin.png)

## $\Delta$AP-Net2

In [94]:
from apnet_pt.pt_datasets.dapnet_ds import dapnet2_module_dataset_apnetStored

delta_energies = df_plot['HF/6-31G* error'].tolist()

# Only operates in pre-batched mode
ds_dap2 = dapnet2_module_dataset_apnetStored(
    root="data_dir",
    r_cut=5.0,
    r_cut_im=8.0,
    spec_type=None,
    max_size=None,
    force_reprocess=True,
    batch_size=2,
    num_devices=1,
    skip_processed=False,
    skip_compile=True,
    print_level=2,
    in_memory=True,
    m1="HF/6-31G*",
    m2="CCSD(T)/CBS",
    qcel_molecules=df_plot['qcel_molecule'].tolist(),
    energy_labels=delta_energies,
)
print(ds_dap2)

⠧ Connecting to=hpc_cpu... status=CONNECTED
Received 86 QCElemental molecules with energy labels
running on the CPU
running on the CPU
Loading pre-trained APNet2_MPNN model from /dev/shm/cybershuttle/envs/18dccede/lib/python3.10/site-packages/apnet_pt/models/ap2_ensemble/ap2_0.pt
self.dataset=None
raw_path: data_dir/raw/splinter_spec1.pkl
Loading dimers...
Creating data objects...
len(qcel_mols)=86, self.batch_size=2


raw_path: data_dir/raw/splinter_spec1.pkl
Loading dimers...
Creating data objects...
len(qcel_mols)=86, self.batch_size=2
self.root='data_dir', self.spec_type=None, self.in_memory=True
raw_path: data_dir/raw/splinter_spec1.pkl
Loading dimers...
Saving to data_dir/processed_delta/targets_HF6-31G_to_CCSD_LP_T_RP_CBS.pt
dapnet2_module_dataset_apnetStored(43)


In [ ]:
from apnet_pt.AtomPairwiseModels.dapnet2 import dAPNet2Model

dap2 = dAPNet2Model(
    atom_model=AtomModel().set_pretrained_model(model_id=0),
    apnet2_model=APNet2Model().set_pretrained_model(model_id=0).set_return_hidden_states(True),
)
dap2.train(
    ds_dap2,
    n_epochs=50,
    skip_compile=True,
    split_percent=0.6,
)

In [ ]:
dAPNet2_ies_predicted_transfer = dap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=2,
)
df_plot['dAPNet2'] = dAPNet2_ies_predicted_transfer
df_plot['HF/6-31G*-dAPNet2'] = df_plot['HF/6-31G*'] - df_plot['dAPNet2']
print(df_plot[['dAPNet2', 'HF/6-31G*', 'HF/6-31G*-dAPNet2',  'reference']])
df_plot['dAPNet2 error'] = (df_plot['HF/6-31G*-dAPNet2'] - df_plot['reference']).astype(float)

error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
        "APNet2 transfer": "APNet2 transfer error",
        "dAPNet2 HF/6-31G* to CCSD(T)/CBS": "dAPNet2 error",
    },
    output_filename="S22-IE-AP2-dAP2.png",
    rcParams={},
    usetex=False,
    figure_size=(6, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

In [ ]:
# Be careful with this for it can corrupt running status...
# !ps aux | grep qcfractal | awk '{ print $2 }' | xargs kill -9

![S22-IE_violin-AP2-dAP2.png](./S22-IE-AP2-dAP2_violin.png)

# The end...